In [1]:
import pandas as pd

##### Imports image dataset ####### 
data = pd.read_csv(r"C:\Users\ASUS\OneDrive - University of the Philippines\Desktop\University of the Philippines\Fourth Year Second Sem\MATH 190\Sir Dataset\Compiled.csv") 

#### array Indexing ###
X = data.iloc[:,1:71].to_numpy() #Data
Y = data.iloc[:,0].to_numpy()

In [2]:
X.shape

(540, 70)

In [94]:
## Embed all of the data into 3 dimension

from sklearn.pipeline import make_pipeline
embedding_dimension = 3
embedding_time_delay = 16
stride = 3
TE = SingleTakensEmbedding(
                parameters_type="fixed",
                n_jobs=2,
                time_delay=embedding_time_delay,
                dimension=embedding_dimension,
                stride=stride,
                )
Takens_emb = np.array([TE.fit_transform(instance) for instance in X])

In [95]:
y_embedded = np.array([instance[None, :, :] for instance in Takens_emb]) 

In [3]:
### Topological Pipeline

##### Imports the necessary modules #######
from sklearn.pipeline import make_pipeline, make_union
from gtda.diagrams import PersistenceEntropy
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Scaler
from gtda.diagrams import Amplitude

homology_dimensions = [0, 1]


###### Persistence diagram acquisition ########
diagram_steps = [
    [
        VietorisRipsPersistence(homology_dimensions=homology_dimensions, n_jobs=6),
        Scaler(n_jobs=-1),
    ]
]

####### list of metric used to obtain the amplitude of PD ########
metric_list = [
    {"metric": "bottleneck", "metric_params": {}},
    {"metric": "wasserstein", "metric_params": {"p": 1}},
    {"metric": "wasserstein", "metric_params": {"p": 2}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 1, "n_bins": 50}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 2, "n_bins": 50}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 1, "n_bins": 50}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 2, "n_bins": 50}},
    {"metric": "betti", "metric_params": {"p": 1, "n_bins": 50}},
    {"metric": "betti", "metric_params": {"p": 2, "n_bins": 50}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 0.15, "n_bins": 50}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 0.15, "n_bins": 50}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 0.15, "n_bins": 50}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 0.15, "n_bins": 50}},
]

####### union of all features as an ordered n-tuples i.e. n-dimensional vector, in our case the resulting feature has 476 dimension #######
feature_union = make_union(
    *[PersistenceEntropy(nan_fill_value=-1)]
    + [Amplitude(**metric, n_jobs=-1) for metric in metric_list]
)

####### Pipeline union. this way a pipeline is analogous to a function that maps every data point to a feature vector########
tda_union = make_union(
    *[make_pipeline(*diagram_step, feature_union) for diagram_step in diagram_steps],
    n_jobs=-1
)
     

In [4]:
#### Displays the pipeline diagram ##### 
from sklearn import set_config
set_config(display='diagram')  

tda_union

FeatureUnion(n_jobs=-1,
             transformer_list=[('pipeline',
                                Pipeline(steps=[('vietorisripspersistence',
                                                 VietorisRipsPersistence(homology_dimensions=[0,
                                                                                              1],
                                                                         n_jobs=6)),
                                                ('scaler', Scaler(n_jobs=-1)),
                                                ('featureunion',
                                                 FeatureUnion(transformer_list=[('persistenceentropy',
                                                                                 PersistenceEntropy(nan_fill_value=-1)),
                                                                                ('amplitude-1',
                                                                                 Amplitude(metric='bottleneck',
                                                                                           metric_para...
                                                                                           metric_params={'n_bins': 50,
                                                                                                          'p': 1,
                                                                                                          'sigma': 0.15},
                                                                                           n_jobs=-1)),
                                                                                ('amplitude-11',
                                                                                 Amplitude(metric='heat',
                                                                                           metric_params={'n_bins': 50,
                                                                                                          'p': 1,
                                                                                                          'sigma': 0.15},
                                                                                           n_jobs=-1)),
                                                                                ('amplitude-12',
                                                                                 Amplitude(metric='heat',
                                                                                           metric_params={'n_bins': 50,
                                                                                                          'p': 2,
                                                                                                          'sigma': 0.15},
                                                                                           n_jobs=-1)),
                                                                                ('amplitude-13',
                                                                                 Amplitude(metric='heat',
                                                                                           metric_params={'n_bins': 50,
                                                                                                          'p': 2,
                                                                                                          'sigma': 0.15},
                                                                                           n_jobs=-1))]))]))])

In [103]:
X_tda = np.array([tda_union.fit_transform(instance) for instance in y_embedded])

In [106]:
X_tda.shape

(540, 1, 28)

In [107]:
X_tda = np.reshape(X_tda, (X_tda.shape[0], X_tda.shape[2])) 

In [108]:
X_tda.shape

(540, 28)

In [110]:
### export to csv as a feature dataset 

X_tda = pd.DataFrame(X_tda)
X_tda.to_csv("Raw Feature.csv", index=False)